# IFRC Appeal Documents Fetcher

This notebook fetches appeal documents from the IFRC GO API based on an appeal code.

## 1. Import Required Libraries

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()
print('Environment variables loaded successfully!')

Environment variables loaded successfully!


## 2. Define Functions

### 2.1 Get Authorization Headers

In [2]:
def get_auth_headers():
    """
    Get authorization headers using token from environment variable.
    
    Returns:
        dict: Headers dictionary with Authorization token
    
    Raises:
        ValueError: If IFRC_API_TOKEN is not found in environment
    """
    auth_token = os.getenv('IFRC_API_TOKEN')
    
    if not auth_token:
        raise ValueError('IFRC_API_TOKEN not found in environment. Please check your .env file.')
    
    return {"Authorization": f"Token {auth_token}"}

### 2.2 Fetch Appeal Data

In [3]:
def fetch_appeal_data(appeal_code):
    """
    Fetch appeal ID from the GO API using an appeal code.
    
    Args:
        appeal_code: The appeal code to look up (e.g., 'MDRCM002')
    
    Returns:
        str: The appeal ID (aid) for the given appeal code
    
    Raises:
        ValueError: If no appeal is found with the given code
    """
    appeal_url = f"https://goadmin.ifrc.org/api/v2/appeal/?code={appeal_code}"
    headers = get_auth_headers()
    
    response = requests.get(appeal_url, headers=headers)
    
    # Check for successful response
    if response.status_code != 200:
        raise Exception(f"API returned status code {response.status_code}: {response.text}")
    
    # Use response.json() directly instead of json.loads(response.text)
    appeal_data = response.json()
    
    # Check if results exist
    results = appeal_data.get("results", [])
    if not results:
        raise ValueError(f"No appeal found with code: {appeal_code}")
    
    appeal_id = results[0].get("aid")
    return appeal_id

### 2.3 Fetch Documents Data

In [4]:
def fetch_documents_data(appeal_id, offset=0, page_size=50):
    """
    Fetch a page of documents for a given appeal ID.
    
    Args:
        appeal_id: The appeal ID to fetch documents for
        offset: Pagination offset (default: 0)
        page_size: Number of documents per page (default: 50)
    
    Returns:
        list: List of document dictionaries from the API response
    """
    documents_url = f"https://goadmin.ifrc.org/api/v2/appeal_document/"
    headers = get_auth_headers()
    
    params = {
        "limit": page_size,
        "offset": offset,
        "appeal": appeal_id
    }
    
    response = requests.get(documents_url, params=params, headers=headers)
    
    # Check for successful response
    if response.status_code != 200:
        print(f"Warning: API returned status code {response.status_code}")
        return []
    
    # Use response.json() directly
    documents_data = response.json()
    return documents_data.get("results", [])

### 2.4 Fetch All Documents

In [5]:
def all_documents(appeal_code):
    """
    Fetch all documents for a given appeal code with pagination handling.
    
    Args:
        appeal_code: The appeal code to fetch documents for (e.g., 'MDRCM002')
    
    Returns:
        pd.DataFrame: DataFrame containing all appeal documents
    """
    # First, get the appeal ID from the appeal code
    appeal_id = fetch_appeal_data(appeal_code)
    print(f"Found appeal ID: {appeal_id} for code: {appeal_code}")

    # Initialize an empty list to store appeal documents
    appeal_documents = []
    offset = 0
    page_size = 50

    # Fetch all documents data from the paginated API endpoint
    while True:
        documents_data = fetch_documents_data(appeal_id, offset=offset, page_size=page_size)

        # Break the loop if there are no more documents
        if not documents_data:
            break

        # Extend the list of appeal documents with documents from the current page
        appeal_documents.extend(documents_data)
        print(f"Fetched {len(documents_data)} documents (total: {len(appeal_documents)})")

        # Increment offset for the next page
        offset += page_size

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(appeal_documents)
    
    return df

## 3. Fetch Appeal Documents

In [6]:
# Specify the appeal code
appeal_code = 'MDRCM002'

# Fetch all documents for the appeal
appeal_documents_df = all_documents(appeal_code)

print(f"\nTotal documents fetched: {len(appeal_documents_df)}")

Found appeal ID: 3388 for code: MDRCM002
Fetched 1 documents (total: 1)

Total documents fetched: 1


## 4. Display Results

In [7]:
# Show the columns in the DataFrame
print("Available columns:")
print(appeal_documents_df.columns.tolist())

Available columns:
['created_at', 'document', 'document_url', 'appeal', 'type', 'iso', 'description', 'id', 'name', 'translation_module_original_language']


In [8]:
# Display the DataFrame
appeal_documents_df

,created_at,document,document_url,appeal,type,iso,description,id,name,translation_module_original_language
0,2021-02-18T00:00:00Z,None,https://www.ifrc.org/docs/Appeals/21/IB%20Guin...,"{'id': 3388, 'code': 'MDRGN012', 'event': None...",Information Bulletin,GN,IB Guinea_Ebola Virus Disease,18504,IB Guinea_Ebola Virus Disease,en
